In [28]:
import numpy as np
import itertools
import pandas as pd

def generate_beta(n, rho):
    rng = np.random.default_rng(seed=None)
    betas = np.where(rng.random(n) < rho, 0.0, rng.normal(size=n))
    return betas

def generate_r_uniform_hypergraph(n, r, beta):
    edges = []
    degrees = np.zeros(n, dtype=int)

    for comb in itertools.combinations(range(n), r):
        weight_sum = beta[list(comb)].sum()
        p = np.exp(weight_sum) / (1 + np.exp(weight_sum))
        if np.random.rand() < p:
            edges.append(comb)
            for v in comb:
                degrees[v] += 1
    return edges, degrees


def add_gaussian_noise(degrees, epsilon, delta, r):
    c_squared = 2 * np.log(1.25 / delta)
    sigma = np.sqrt(c_squared) * np.sqrt(r) / epsilon
    noise = np.random.normal(0, sigma, size=degrees.shape)
    return degrees + noise

def stable_sigmoid(x):
    out = np.empty_like(x)
    pos = x >= 0
    neg = ~pos
    out[pos] = 1 / (1 + np.exp(-x[pos]))
    exp_x = np.exp(x[neg])
    out[neg] = exp_x / (1 + exp_x)
    return out

def gradient_descent(n, r, noisy_degrees, lr, max_iter, tol=1e-6, verbose=True):
    beta = np.zeros(n)
    combs = np.array(list(itertools.combinations(range(n), r)))  # shape (C, r)
    print(f"learning rate = {lr}")

    for t in range(max_iter):
        beta_sums = np.sum(beta[combs], axis=1)
        sigm = stable_sigmoid(beta_sums)
        grad = np.zeros(n)
        for j in range(r):
            np.add.at(grad, combs[:, j], sigm)

        grad -= noisy_degrees
        grad_norm = np.linalg.norm(grad)
        if verbose and t % 100 == 0:
            print(f"Iter {t}: {grad_norm:.4f}", end = "-> ")

        if grad_norm < tol:
            break
        beta -= lr * grad

    if grad_norm > 0.001 and lr > 0.00001:
        print("Restarting..")
        gradient_descent(n, r, noisy_degrees, lr/10, max_iter)
        
    return beta

def run_simulation(n, r, rho, delta):
    beta = generate_beta(n, rho)
    edges, degrees = generate_r_uniform_hypergraph(n, r, beta)
    # print("Degree generation done!")

    lr = 0.1
    max_iter = 3000
    
    # beta_hat_0 = gradient_descent(n, r, degrees, lr, max_iter)
    # l2_error = [np.linalg.norm(beta - beta_hat_0)]
    # linf_error = [np.abs(beta - beta_hat_0).max()]

    e_list = [0.1, 0.5, 1, 1.5, 2]
    for epsilon in e_list:
        noisy_degrees = add_gaussian_noise(degrees, epsilon, delta, r)
        beta_hat = gradient_descent(n, r, noisy_degrees, lr, max_iter)
        l2_error.append(np.linalg.norm(beta - beta_hat))
        linf_error.append(np.abs(beta - beta_hat).max())
    
    return l2_error, linf_error

delta = 1e-5
rho = 0.6

n = 100
# n = 250
# n = 500
# n = 750
# n = 1000

r = 3
# r = 4
# r = 6

print(f"Running for r = {r}, n = {n} -> ", end = " ")
l2 = np.zeros((20,6))
linf = np.zeros((20,6))
# for t in range(20):
#     l2[t], linf[t] = run_simulation(n, r, rho, delta)
#     print(t, end = " ")
# print(" ")

# columns = ["non-private", "0.1", "0.5", "1.0", "1.5", "2.0"]
# df1 = pd.DataFrame(l2, columns=columns)
# df2 = pd.DataFrame(linf, columns=columns)
# df1.to_csv(f"l_2 error, n - {n}, r - {r}", index=False)
# df2.to_csv(f"l_inf error, n - {n}, r - {r}", index=False)

l2 = np.zeros((20,6))
linf = np.zeros((20,6))

# Open files for writing
# with open(f"l_2 error, n - {n}, r - {r}.csv", 'w') as f1, open(f"l_inf error, n - {n}, r - {r}.csv", 'w') as f2:
#     # Write headers
#     columns = ["non-private", "0.1", "0.5", "1.0", "1.5", "2.0"]
#     f1.write(','.join(columns) + '\n')
#     f2.write(','.join(columns) + '\n')
    
for t in range(20):
    l2[t], linf[t] = run_simulation(n, r, rho, delta)
    print(t, end = " ")
        
#         # Write current row to both files
#         f1.write(','.join(map(str, l2[t])) + '\n')
#         f2.write(','.join(map(str, linf[t])) + '\n')

Running for r = 3, n = 100 ->  learning rate = 0.1
Iter 0: 6526.5403-> Iter 100: 18598.8616-> Iter 200: 15440.4022-> Iter 300: 18897.7205-> Iter 400: 17433.7522-> Iter 500: 15919.4797-> Iter 600: 19463.3816-> Iter 700: 16242.7046-> Iter 800: 16905.1664-> Iter 900: 19135.8139-> Iter 1000: 15485.2394-> Iter 1100: 18446.7333-> Iter 1200: 17875.5692-> Iter 1300: 15723.9627-> Iter 1400: 19250.5109-> Iter 1500: 16673.2077-> Iter 1600: 16425.2542-> Iter 1700: 19479.4705-> Iter 1800: 15735.2024-> Iter 1900: 17785.3119-> Iter 2000: 18360.5719-> Iter 2100: 15533.3572-> Iter 2200: 19016.9690-> Iter 2300: 17186.6394-> Iter 2400: 16050.4249-> Iter 2500: 19535.4835-> Iter 2600: 16097.2632-> Iter 2700: 17082.5095-> Iter 2800: 18992.2489-> Iter 2900: 15453.8715-> Restarting..
learning rate = 0.01
Iter 0: 6526.5403-> Iter 100: 17453.5790-> Iter 200: 16769.4041-> Iter 300: 18217.3448-> Iter 400: 16072.1213-> Iter 500: 18980.7273-> Iter 600: 15676.5223-> Iter 700: 19330.2611-> Iter 800: 15522.7062-> Iter

KeyboardInterrupt: 